In [2]:
import pandas as pd  # Data handling
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Basic plotting
import seaborn as sns  # Advanced visualization
from sklearn.model_selection import train_test_split  # Splitting data
from sklearn.preprocessing import StandardScaler  # Normalization
# Import necessary metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Import machine learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [3]:
df_2018 = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\CIC_IDS_2018.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_16276\826868802.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\CIC_IDS_2018.csv")


In [4]:
df_2017 = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\CIC_IDS_2017.csv")

In [7]:
df_unsw = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\UNSW_NB15.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_16276\1858947473.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unsw = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\UNSW_NB15.csv")


In [9]:
df_2017.columns = df_2017.columns.str.strip()
df_2018.columns = df_2018.columns.str.strip()
df_unsw.columns = df_unsw.columns.str.strip()



In [15]:
label_map_2017 = {
    "BENIGN": "Benign",
    "DoS Hulk": "DoS",
    "PortScan": "PortScan",
    "DDoS": "DDoS",
    "DoS GoldenEye": "DoS",
    "FTP-Patator": "BruteForce",
    "SSH-Patator": "BruteForce",
    "DoS slowloris": "DoS",
    "DoS Slowhttptest": "DoS",
    "Bot": "Botnet",
    "Web Attack � Brute Force": "Web Attack",
    "Web Attack � XSS": "Web Attack",
    "Web Attack � Sql Injection": "Web Attack",
    "Infiltration": "Infiltration",
    "Heartbleed": "Heartbleed"
}

label_map_2018 = {
    "Benign": "Benign",
    "DDOS attack-HOIC": "DDoS",
    "Bot": "Botnet",
    "FTP-BruteForce": "BruteForce",
    "SSH-Bruteforce": "BruteForce",
    "Infilteration": "Infiltration",  
    "DoS attacks-GoldenEye": "DoS",
    "DoS attacks-Slowloris": "DoS",
    "DDOS attack-LOIC-UDP": "DDoS",
}
df_2017["Label"] = df_2017["Label"].map(label_map_2017)
df_2018["Label"] = df_2018["Label"].map(label_map_2018)

# Drop unmapped labels (e.g., "Label" in CICIDS2018)
df_2017 = df_2017.dropna(subset=["Label"])
df_2018 = df_2018.dropna(subset=["Label"])


In [27]:
cols_to_drop = ["Protocol", "Timestamp"]
df_2018 = df_2018.drop(columns=cols_to_drop, errors="ignore")
non_numeric_cols = ["Label"]
for df in [df_2018]:
    for col in df.columns:
        if col not in non_numeric_cols and df[col].dtype == "object":
            df[col] = pd.to_numeric(df[col], errors="coerce")


In [44]:
# Assign binary labels
df_2017["Binary_Label"] = df_2017["Label"].apply(lambda x: 0 if x == "Benign" else 1)
df_2018["Binary_Label"] = df_2018["Label"].apply(lambda x: 0 if x == "Benign" else 1)


In [35]:

column_mapping = {
    # 2017 Columns               # 2018 Columns
    'Destination Port':          'Dst Port',
    'Total Fwd Packets':         'Tot Fwd Pkts',
    'Total Backward Packets':    'Tot Bwd Pkts',
    'Total Length of Fwd Packets': 'TotLen Fwd Pkts',
    'Total Length of Bwd Packets': 'TotLen Bwd Pkts',
    'Fwd Packet Length Max':     'Fwd Pkt Len Max',
    'Fwd Packet Length Min':     'Fwd Pkt Len Min',
    'Fwd Packet Length Mean':    'Fwd Pkt Len Mean',
    'Fwd Packet Length Std':     'Fwd Pkt Len Std',
    'Bwd Packet Length Max':     'Bwd Pkt Len Max',
    'Bwd Packet Length Min':     'Bwd Pkt Len Min',
    'Bwd Packet Length Mean':    'Bwd Pkt Len Mean',
    'Bwd Packet Length Std':     'Bwd Pkt Len Std',
    'Flow Bytes/s':              'Flow Byts/s',
    'Flow Packets/s':            'Flow Pkts/s',
    'Fwd Packets/s':             'Fwd Pkts/s',
    'Bwd Packets/s':             'Bwd Pkts/s',
    'Min Packet Length':         'Pkt Len Min',
    'Max Packet Length':         'Pkt Len Max',
    'Packet Length Mean':        'Pkt Len Mean',
    'Packet Length Std':         'Pkt Len Std',
    'Packet Length Variance':    'Pkt Len Var',
    'FIN Flag Count':            'FIN Flag Cnt',
    'SYN Flag Count':            'SYN Flag Cnt',
    'RST Flag Count':            'RST Flag Cnt',
    'PSH Flag Count':            'PSH Flag Cnt',
    'ACK Flag Count':            'ACK Flag Cnt',
    'URG Flag Count':            'URG Flag Cnt',
    'CWE Flag Count':            'CWE Flag Count',
    'ECE Flag Count':            'ECE Flag Cnt',
    'Average Packet Size':       'Pkt Size Avg',
    'Avg Fwd Segment Size':      'Fwd Seg Size Avg',
    'Avg Bwd Segment Size':      'Bwd Seg Size Avg',
    'Fwd Header Length.1':       'Fwd Header Len',
    'Fwd Avg Bytes/Bulk':        'Fwd Byts/b Avg',
    'Fwd Avg Packets/Bulk':      'Fwd Pkts/b Avg',
    'Fwd Avg Bulk Rate':         'Fwd Blk Rate Avg',
    'Bwd Avg Bytes/Bulk':        'Bwd Byts/b Avg',
    'Bwd Avg Packets/Bulk':      'Bwd Pkts/b Avg',
    'Bwd Avg Bulk Rate':         'Bwd Blk Rate Avg',
    'Subflow Fwd Packets':       'Subflow Fwd Pkts',
    'Subflow Fwd Bytes':         'Subflow Fwd Byts',
    'Subflow Bwd Packets':       'Subflow Bwd Pkts',
    'Subflow Bwd Bytes':         'Subflow Bwd Byts',
    'Init_Win_bytes_forward':    'Init Fwd Win Byts',
    'Init_Win_bytes_backward':   'Init Bwd Win Byts',
    'act_data_pkt_fwd':          'Fwd Act Data Pkts',
    'min_seg_size_forward':      'Fwd Seg Size Min'
}

# Rename 2017 columns to match 2018 naming convention
df_2017 = df_2017.rename(columns=column_mapping)

In [58]:
df_2018.dropna(inplace=True)
df_2017.dropna(inplace=True)





In [39]:
# 1. Update column mapping
column_mapping.update({
    'Bwd IAT Total': 'Bwd IAT Tot',
    'Fwd IAT Total': 'Fwd IAT Tot',
    'Bwd Header Length': 'Bwd Header Len'
})

# 2. Rename 2017 columns
df_2017 = df_2017.rename(columns=column_mapping)

# 3. Drop columns that exist only in 2017 and couldn't be mapped
df_2017 = df_2017.drop(columns=['Fwd Header Length'], errors='ignore')

# 4. Verify no remaining mismatches
unique_to_2017 = set(df_2017.columns) - set(df_2018.columns)
unique_to_2018 = set(df_2018.columns) - set(df_2017.columns)

print("Remaining unique to 2017:", unique_to_2017)  # Should be empty
print("Remaining unique to 2018:", unique_to_2018)  # Should be empty

Remaining unique to 2017: set()
Remaining unique to 2018: set()


In [43]:
merged_df = pd.concat([df_2017, df_2018], ignore_index=True)


In [51]:
# Shuffle the DataFrame while maintaining data integrity
shuffled_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [53]:
shuffled_df['is_attack'] = shuffled_df['Label'].apply(
    lambda x: 0 if x == 'Benign' else 1  # Encode directly as integers
)

In [61]:
shuffled_df.to_csv(r"C:\Users\hp\Desktop\TFG\Datasets\CIC_IDS_Merged.csv", index=False)